<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/SpeechRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech_Recognition ライブラリによる音声認識の例

## 1. 準備

### (1) ソフトのインストール

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

### (2) ライブラリのインストール

In [ ]:
# SpeechRecognitionのinstall
!pip install speechrecognition
# pyaudioのinstall
!pip install Pyaudio

### (3) ブラウザからマイクを使う設定


In [3]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

def record_s(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  return s

def record_save(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  audio.export("audio.wav",format="wav" )

## 2. 音声認識

In [ ]:
import speech_recognition as sr
import subprocess
import tempfile
 
r = sr.Recognizer()

In [ ]:
# 音声入力->認識
while True:

    print("何かお話しして下さい。")
    record_save(5) # 録音時間
    with sr.AudioFile("audio.wav") as source: 
        audio = r.listen(source)
    try:
        # Google Web Speech APIで音声認識
        text = r.recognize_google(audio, language="ja-JP") # 言語設定変更可能
    except sr.UnknownValueError:
        print("　　Google Web Speech APIは音声を認識できませんでした。")
    except sr.RequestError as e:
        print("　　GoogleWeb Speech APIに音声認識を要求できませんでした;"
              " {0}".format(e))
    else:
        print("　　認識結果：" + text)
    if text == "終わりです":
        break
print("終了")

## 3. 応用

In [ ]:
#Chromiumとseleniumをインストール
print("前処理を開始")
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

#ライブラリをインポート
from selenium import webdriver
import time

#---------------------------------------------------------------------------------------
# 処理開始
#---------------------------------------------------------------------------------------
# ブラウザをheadlessモード実行
print("\nブラウザを設定")
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

In [ ]:
# サイトにアクセス
print("サイトにアクセス開始")
driver.get("https://www.mod.go.jp/nda/")
time.sleep(3)

# driver.find_elements_by_css_selector("xxx") 的な処理を自由に
print("サイトのタイトル：", driver.title)

print("\nお疲れさまです。\n処理が完了しました。")